In [1]:
import ee
import geemap

In [2]:
L8bands = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

In [3]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [5]:
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to TOA reflectance, without the QA bands.
    return image.updateMask(mask).divide(10000).select(L8bands).copyProperties(image, ["system:time_start"])

In [7]:
# Creating an add variable function for Landsat 8 index calculation.
def compute_indices(image):
    NDVI = image.expression('(B5-B4)/(B5+B4)', 
                            {'B4': image.select('B4'),
                             'B5': image.select('B5')}).rename('ndvi')
    MNDWI = image.expression('(B3-B6) / (B3+B6)',
                             {'B3': image.select('B3'),
                              'B6': image.select('B6')}).rename('mndwi')
    EVI = image.expression('2.5 * ((B5 - B4)/(B5 +(6*B4)-(7.5*B2)+1))',
                           {'B2': image.select('B2'),
                            'B4': image.select('B4'),
                            'B5': image.select('B5')}).rename('evi')
    LSWI = image.expression('(B5-B6)/(B5+B6)', 
                            {'B5': image.select('B5'),
                             'B6': image.select('B6')}).rename('lswi')
    return image.addBands(NDVI).addBands(MNDWI).addBands(EVI).addBands(LSWI)

In [7]:
point = ee.Geometry.Point([106.4943, 20.4858])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2016-01-01', '2020-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(bottom=1826.0, center=[20.485800000000005, 106.49430000000001], controls=(WidgetControl(options=['position…

In [4]:
# ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

In [5]:
# image.get('CLOUD_COVER').getInfo()

In [11]:
image2 = compute_indices(image)

In [3]:
# ee.Date(image2.get('system:time_start')).format('YYYY-MM-dd').getInfo()

In [2]:
# image2.get('CLOUD_COVER').getInfo()

In [1]:
# image2.getInfo()

In [28]:
image3 = image2.select('ndvi', 'mndwi', 'evi', 'lswi')

In [13]:
# Add the boundary of Thai Binh province
thaibinh_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Thai Binh/VungTB.shp'
thaibinh = geemap.shp_to_ee(thaibinh_shp)

In [14]:
# Show the boundary on map
Map.addLayer(thaibinh, {},"Tinh Thai Binh")
Map.centerObject(thaibinh, 10)
Map

Map(bottom=14851.0, center=[20.468675714429068, 106.53894403023824], controls=(WidgetControl(options=['positio…

In [15]:
# Add training markers from shape files
agriculture_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Markers/agriculture_markers.shp'
evergreen_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Markers/evergreen_markers.shp'
mangrove_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Markers/mangrove_markers.shp'
others_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Markers/other_markers.shp'
residence_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Markers/residence_markers.shp'
shrimp_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Markers/shrimp_markers.shp'
water_shp = 'D:/@MASTER THESIS/@Programming/@GGE/Thuy/Markers/water_markers.shp'
# Shape files to ee objects
cl_agriculture = geemap.shp_to_ee(agriculture_shp)
cl_evergreen = geemap.shp_to_ee(evergreen_shp)
cl_mangrove = geemap.shp_to_ee(mangrove_shp)
cl_others = geemap.shp_to_ee(others_shp)
cl_residence = geemap.shp_to_ee(residence_shp)
cl_shrimp = geemap.shp_to_ee(shrimp_shp)
cl_water = geemap.shp_to_ee(water_shp)
# Make markers collections
classNames = cl_agriculture.merge(cl_evergreen).merge(cl_mangrove).merge(cl_others). \
                            merge(cl_residence).merge(cl_shrimp).merge(cl_water)
# Show the markers on map
Map.addLayer(classNames, {}, 'Markers')
# Map

In [39]:
L8sr = L8sr.median().clip(thaibinh)

In [36]:
# # Compute multiple indices
L8col = compute_indices(L8sr)
# print(L8col)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [37]:
# L8col.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -3.2768001556396484,
    'max': 3.276700019836426},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -3.2768001556396484,
    'max': 3.276700019836426},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -3.2768001556396484,
    'max': 3.276700019836426},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -3.2768001556396484,
    'max': 3.276700019836426},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': -3.2768001556396484,
    'max': 3.276700019836426},
   'crs

In [8]:
# Set the vis
thaibinh_vis = {
    'bands': ['B4', 'B3', 'B2'], 
    'min': 0,
    'max': 0.3
    #'gamma': 1.5
}

# Display the results.
Map.addLayer(L8sr, thaibinh_vis, 'Thai Banh LC8')
Map

Map(bottom=29232.0, center=[20.485800000000005, 106.49430000000001], controls=(WidgetControl(options=['positio…

In [30]:
# Set bands and label for classifier
bands = ['ndvi', 'mndwi', 'evi', 'lswi']
label = 'landcover'

In [32]:
# classify training data by randomForest
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [33]:
# Run the classification
classified = image3.select(bands).classify(classifier)

In [41]:
# Display classification
cl_vis_params = {
    'min': 1, 
    'max': 7, 
    'palette': ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']
}

legend_keys = ['Mangrove', 'Shrimp', 'Residence', 'Water','Agriculture','Other','Evergreen']
legend_colors = ['#98ff00','#0b4a8b','#ffc82d','#00ffff','#bf04c2','#ff0000','#008800']

Map.centerObject(thaibinh, 10)
Map.addLayer(classified, cl_vis_params, 'classification')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

Map(bottom=231791.0, center=[20.468675714429068, 106.53894403023824], controls=(WidgetControl(options=['positi…